In [1]:
!pip install geopandas
!pip install folium
!pip install elevation
!pip install pyproj
!pip install mplstereonet


     |████████████████████████████████| 71kB 1.3MB/s  eta 0:00:01
ERROR: papermill 1.2.1 has requirement tqdm>=4.32.2, but you'll have tqdm 4.26.0 which is incompatible.
  Found existing installation: Click 7.0
    Uninstalling Click-7.0:
      Successfully uninstalled Click-7.0


In [2]:
#from: http://geologyandpython.com/dem-processing.html
# https://gis.stackexchange.com/questions/116319/plotting-elevation-maps-and-shaded-relief-images-from-latitude-longitude-and-e
import numpy as np
import pandas as pd
import geopandas as gpd
import pyproj
import folium
from shapely import geometry
import matplotlib.pyplot as plt
import elevation
import os
import datetime
%matplotlib inline

In [3]:
import plotly.plotly as py
import plotly.graph_objs as go




ImportError: 
The plotly.plotly module is deprecated,
please install the chart-studio package and use the
chart_studio.plotly module instead. 


In [ ]:
import pandas as pd
import numpy as np
import scipy.linalg
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import datetime

import mplstereonet
#from mpl_toolkits.basemap import pyproj
#from mpl_toolkits.basemap import Basemap
from pyproj import Proj, transform
###################################plotly_block###################################################### 
# Import plotly package online####################
import plotly
plotly.tools.set_credentials_file(username='davide.schenone', api_key='WBSgVI0FXgSIRecK2cHU')
import plotly.graph_objs as go
# Check ploltly version
plotly.__version__

#plotly offline###############
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *
init_notebook_mode(connected=True) #attivando o disattivando questa riga si ottiene un plot 
                                   #contestuale rispettivamente in notebook oppure su un html

# To communicate with Plotly's server, sign in with credentials file
import plotly.plotly as py
################################################################################################

from pandas import ExcelWriter
from pandas import ExcelFile
# Create data with x and y random over [-2, 2], and z a Gaussian function of x and y.
#np.random.seed(12345)
#x = 2 * (np.random.random(500) - 0.5)
#y = 2 * (np.random.random(500) - 0.5)

#def f(x, y):
 #   return np.exp(-(x + y ** 2))

#z = f(x, y)

#imposto i dati
#importa il file originale
#df = pd.read_csv('Saorge_Taggia_erh_erz_prof.csv', sep='\t', names=['y', 'x','z','a','b'])

#importa il file convertito con convergo
df = pd.read_excel('terremoti_santo.xlsx', sheet_name='Foglio1')


df['z'] = df['Prof corr']  #creo il campo z profondità con valori negativi


df['Data'] = df['Data'].astype(str).str.zfill(6) #riempie la mancanza dello zero nei primi valori
df['Data'] =  pd.to_datetime(df['Data'],  format='%y%m%d')
#df['Ora'] =  pd.to_time(df['Ora'],  format='%H%M')
#creo una colonna con  la differenza in giorni tra la data e una posta abbastanza indietro nel passato
df['Deltatime'] = df['Data'] - datetime.date(1989,1,1)
df['Deltatime'] = df['Deltatime'].astype(datetime.timedelta).map(lambda x: np.nan if pd.isnull(x) else x.days)

#converto le coordinate wgs 84 in UTM epsg 3003 Gauss-Boaga 
x1 = np.array(df['Longit.'])
y1 = np.array(df['Latitud.'])


inProj = Proj(init='epsg:4326')
outProj = Proj(init='epsg:3003')
x2,y2 = transform(inProj,outProj, x1, y1)

df['x'] = x2
df['y'] = y2

#con questo posso fare dei cluster temporali
#IMPOSTO I FILTRI

#df_filter = df.query('Erh<1 and Erz<4') #imposto i parametri di filtro sugli errori orizzontali e verticali


#df_filter = df.query('<Deltatime<') #imposto il filtro sulla data

#imposto i pesi per il calcolo del piano da utilizzare in caso di regressione lineare pesata

df['weight_calc'] = df['Nf']/24 - 0.25
def weight(df):
    if df['Nf'] <=6 :
        return 0
    
    elif  df['Nf'] > 6 and df['Nf']  < 30   :
        return df['weight_calc']
            
    elif  df['Nf'] >= 30 :
        return 1

df['weights'] =  df.apply(weight, axis=1) 
 

    
df_filter = df


coords = df_filter.as_matrix(columns=['Deltatime'])

from sklearn.cluster import DBSCAN
import numpy as np

db = DBSCAN(eps=70, min_samples=10).fit(coords)
labels = db.labels_
from collections import Counter


df_filter = df_filter.assign(Classe = labels)
print(Counter(labels))
#filtro la label di interesse
df_cluster = df_filter[df_filter['Classe'] == 0]



df_filter_2 = df_cluster
coords_2 = df_cluster.as_matrix(columns=['x','y','z'])

from sklearn.cluster import DBSCAN
import numpy as np
#base eps 3000 min_samples 5
db = DBSCAN(eps=1500, min_samples=8).fit(coords_2)
labels_2 = db.labels_
from collections import Counter


df_filter_2 = df_filter_2.assign(Classe_spaziale = labels_2)
print(Counter(labels_2))

#filtro la label di interesse
df_cluster_2 = df_filter_2[df_filter_2['Classe_spaziale'] == 0]



x = df_cluster_2['x']  
    
y = df_cluster_2['y']   
    
z = df_cluster_2['z']


In [ ]:
from mpl_toolkits.mplot3d.axes3d import *
import matplotlib.pyplot as plt
from matplotlib import cm
fig = plt.figure()
ax = Axes3D(fig)
ax.plot_surface(X, Y, Z, rstride=1, cstride=1, cmap=cm.jet,linewidth=1, antialiased=True)
plt.show()

In [ ]:
trace2 = go.Surface(z=Z, x=X, y=Y, colorscale='Earth’', opacity=0.7)

# Package the trace dictionary into a data object
data_test1 = go.Data([trace2])

# Dictionary of style options for all axes
axis = dict(
    showbackground=True, # show axis background
    backgroundcolor="rgb(204, 204, 204)", # set background color to grey
    gridcolor="rgb(255, 255, 255)",       # set grid line color
    zerolinecolor="rgb(255, 255, 255)",   # set zero grid line color
)

# Make a layout object
layout = go.Layout(
    title='1st-order (linear) plane', # set plot title
    scene=go.Scene(  # axes are part of a 'scene' in 3d plots
        xaxis=go.XAxis(axis), # set x-axis style
        yaxis=go.YAxis(axis), # set y-axis style
        zaxis=go.ZAxis(axis)),  # set z-axis style
)

# Make a figure object
fig = go.Figure(data=data_test1, layout=layout)

# Send to Plotly and show in notebook
py.iplot(fig, filename='DTM')

# best-fit quadratic curve (2nd-order)

In [ ]:
# best-fit quadratic curve (2nd-order)
data =  df_cluster_2.as_matrix(columns=['x','y','z'])
# regular grid covering the domain of the data
mn = np.min(data, axis=0)
mx = np.max(data, axis=0)
X,Y = np.meshgrid(np.linspace(mn[0], mx[0], 20), np.linspace(mn[1], mx[1], 20))
XX = X.flatten()
YY = Y.flatten()

trace1 = go.Scatter3d(
    x=data[:,0],
    y=data[:,1],
    z=data[:,2],
    mode='markers',
    marker=dict(size=4, color='red', line=dict(color='black', width=0.5), opacity=0.85)
)



A = np.c_[np.ones(data.shape[0]), data[:,:2], np.prod(data[:,:2], axis=1), data[:,:2]**2]
C,_,_,_ = scipy.linalg.lstsq(A, data[:,2])
    
# evaluate it on a grid
Z = np.dot(np.c_[np.ones(XX.shape), XX, YY, XX*YY, XX**2, YY**2], C).reshape(X.shape)

# plot points and fitted surface using Matplotlib
fig = plt.figure(figsize=(10, 10))
ax = fig.gca(projection='3d')
ax.plot_surface(X, Y, Z, rstride=1, cstride=1, alpha=0.2)
ax.scatter(data[:,0], data[:,1], data[:,2], c='r', s=50)
plt.xlabel('X')
plt.ylabel('Y')
ax.set_zlabel('Z')
ax.axis('equal')
ax.axis('tight')

# plot points and fitted surface using Plotly
trace3 = go.Surface(
    z=Z,  
    x=X, 
    y=Y,
    colorscale='RdBu',
    opacity=0.55
)

# Package the trace dictionary into a data object
data_test3 = go.Data([trace1, trace3])

# Make a layout object
layout = go.Layout(
    title='2nd-order (quadratic) surface', # set plot title
    scene=go.Scene(  # axes are part of a 'scene' in 3d plots
        xaxis=go.XAxis(axis), # set x-axis style
        yaxis=go.YAxis(axis), # set y-axis style
        zaxis=go.ZAxis(axis)),  # set z-axis style
)

# Make a figure object
fig = go.Figure(data=data_test3, layout=layout)

# Send to Plotly and show in notebook
py.iplot(fig, filename='test2')